# Extra Ascent Action Examples
Extra Ascent action examples to showcase different capabilities. See Ascent's [Pipelines](https://ascent.readthedocs.io/en/latest/Actions/Pipelines.html) docs for deeper details on Pipelines.

In [73]:
# ascent + conduit imports
import conduit
import conduit.blueprint
import ascent

import numpy as np

# cleanup any old results
!./cleanup.sh

### Create an example mesh to feed to Ascent

In [74]:
# create example mesh using the conduit blueprint braid helper
mesh = conduit.Node()
conduit.blueprint.mesh.examples.braid("hexs",
                                      25,
                                      25,
                                      25,
                                      mesh)

## Extra Example 1:
### Vector Component filter to extract scalar component of a vector field

In [75]:
a = ascent.Ascent()
a.open()

# publish our mesh to ascent
a.publish(mesh);

# setup actions
actions = conduit.Node()
add_act = actions.append()
add_act["action"] = "add_pipelines"
pipelines = add_act["pipelines"]

# create a  pipeline (pl1) with a vector component filter (f1)
pipelines["pl1/f1/type"] = "vector_component"

# extract vector component params
vec_comp_params = pipelines["pl1/f1/params"]
vec_comp_params["field"] = "vel"
vec_comp_params["component"] = 0
vec_comp_params["output_name"] = "my_comp_vec"

# declare a scene to render the pipeline result

add_act2 = actions.append()
add_act2["action"] = "add_scenes"
scenes = add_act2["scenes"]

# add a scene (s1) with one pseudocolor plot (p1) that 
# will render the result of our pipeline (pl1)
scenes["s1/plots/p1/type"] = "pseudocolor"
scenes["s1/plots/p1/pipeline"] = "pl1"
scenes["s1/plots/p1/field"] = "my_comp_vec"
# set the output file name (ascent will add ".png")
scenes["s1/image_name"] = "out_pipeline_vec_comp_0"

# print our full actions tree
print(actions.to_yaml())

# execute the actions
a.execute(actions)


- 
  action: "add_pipelines"
  pipelines: 
    pl1: 
      f1: 
        type: "vector_component"
        params: 
          field: "vel"
          component: 0
          output_name: "my_comp_vec"
- 
  action: "add_scenes"
  scenes: 
    s1: 
      plots: 
        p1: 
          type: "pseudocolor"
          pipeline: "pl1"
          field: "my_comp_vec"
      image_name: "out_pipeline_vec_comp_0"



In [76]:
# show the resulting image
ascent.jupyter.AscentViewer(a).show()

In [77]:
#close ascent
a.close()

## Extra Example 2:
### Create two renders for one scene, one of them specifying all camera parameters

In [78]:
a = ascent.Ascent()
a.open()

# publish mesh to ascent
a.publish(mesh);

# setup actions
actions = conduit.Node()

#  declare a scene to render 
add_act = actions.append()
add_act["action"] = "add_scenes"
scenes = add_act["scenes"]

# add a scene (s1) with one volume plot (p1)  
scenes["s1/plots/p1/type"] = "volume"
scenes["s1/plots/p1/field"] = "braid"
# change color table
scenes["s1/plots/p1/color_table/name"] = "blue"
# add color table control points 
control_points = scenes["s1/plots/p1/color_table/control_points"]
# control point 1
control_points1 = control_points.append()
control_points1["type"] = "rgb"
control_points1["position"] = 0.0
color1 = np.array([1.0,0.0,0.0],dtype=np.float32)
control_points1["color"].set(color1)
# control point 2
control_points2 = control_points.append()
control_points2["type"] = "rgb"
control_points2["position"] = 0.5
color2 = np.array([0.0,1.0,0.0],dtype=np.float32)
control_points2["color"].set(color2)
# control point 3
control_points3 = control_points.append()
control_points3["type"] = "rgb"
control_points3["position"] = 1.0
color3 = np.array([1.0,1.0,1.0],dtype=np.float32)
control_points3["color"].set(color3)
# set alpha control point 1
control_points4 = control_points.append()
control_points4["type"] = "alpha"
control_points4["position"] = 0.0
control_points4["alpha"] = 0.0
# set alpha control point 2
control_points5 = control_points.append()
control_points5["type"] = "alpha"
control_points5["position"] = 1.0
control_points5["alpha"] = 1.0

# set the output file name -- ignored due to specifying renders
scenes["s1/image_name"] = "out_pipeline_color_test"
# add renders to specify params
renders = scenes["s1/renders"]
# set render 1 (r1) simple
renders["r1/image_width"] = 512 
renders["r1/image_height"] = 512 
renders["r1/image_prefix"] = "out_extra_renders_r1_simple"
# set render 2 (r2) with all camera params
renders["r2/image_width"] = 400 
renders["r2/image_height"] = 400 
renders["r2/image_prefix"] = "out_extra_renders_r2_cam_params"
# set camera params for r2
r2_camera = renders["r2/camera"]
look_at = np.array([1.0,1.0,1.0],dtype=np.float64)
r2_camera["look_at"].set(look_at)
pos = np.array([0.0,25.0,15.0],dtype=np.float64)
r2_camera["position"].set(pos)
up = np.array([0.0,-1.0,0.0],dtype=np.float64)
r2_camera["up"].set(up)
r2_camera["fov"] = 60.0
r2_camera["xpan"] = 0.0
r2_camera["ypan"] = 0.0
r2_camera["zoom"] = 1.0
r2_camera["azimuth"] = 10.0
r2_camera["elevation"] = -10.0
r2_camera["near_plane"] = 0.1
r2_camera["far_plane"] = 100.1

# print our full actions tree
print(actions.to_yaml())

# execute
a.execute(actions)


- 
  action: "add_scenes"
  scenes: 
    s1: 
      plots: 
        p1: 
          type: "volume"
          field: "braid"
          color_table: 
            name: "blue"
            control_points: 
              - 
                type: "rgb"
                position: 0.0
                color: [1.0, 0.0, 0.0]
              - 
                type: "rgb"
                position: 0.5
                color: [0.0, 1.0, 0.0]
              - 
                type: "rgb"
                position: 1.0
                color: [1.0, 1.0, 1.0]
              - 
                type: "alpha"
                position: 0.0
                alpha: 0.0
              - 
                type: "alpha"
                position: 1.0
                alpha: 1.0
      image_name: "out_pipeline_color_test"
      renders: 
        r1: 
          image_width: 512
          image_height: 512
          image_prefix: "out_extra_renders_r1_simple"
        r2: 
          image_width: 400
          image_height: 40

In [79]:
# show the resulting image
ascent.jupyter.AscentViewer(a).show()

In [80]:
# close ascent
a.close()

## Extra Example 3:
### Apply and render a histogram-based sampling filter

In [81]:
a = ascent.Ascent()
a.open()

# publish mesh to ascent
a.publish(mesh);

# setup actions
actions = conduit.Node()
add_act = actions.append()
add_act["action"] = "add_pipelines"
pipelines = add_act["pipelines"]

# create our first pipeline (pl1) 
# with a histsampling filter (f1)
pipelines["pl1/f1/type"] = "histsampling"
# sample the braid field values
# using 32 bins and sample rate of 0.1
histsamp_params = pipelines["pl1/f1/params"]
#histsamp_params["field"] = "braid"
histsamp_params["field"] = "radial"
histsamp_params["sample_rate"] = 0.1
histsamp_params["bins"] = 32

# declare a scene to render our pipeline results
add_act2 = actions.append()
add_act2["action"] = "add_scenes"
scenes = add_act2["scenes"]

# plot (p1) to render our first pipeline (pl1)
scenes["s1/plots/p1/type"] = "pseudocolor"
scenes["s1/plots/p1/pipeline"] = "pl1"
#scenes["s1/plots/p1/field"] = "radial"
scenes["s1/plots/p1/field"] = "braid"

# set the output file name (ascent will add ".png")
scenes["s1/image_name"] = "out_extra_histsampling"

# print our full actions tree
print(actions.to_yaml())

# execute
a.execute(actions)


- 
  action: "add_pipelines"
  pipelines: 
    pl1: 
      f1: 
        type: "histsampling"
        params: 
          field: "radial"
          sample_rate: 0.1
          bins: 32
- 
  action: "add_scenes"
  scenes: 
    s1: 
      plots: 
        p1: 
          type: "pseudocolor"
          pipeline: "pl1"
          field: "braid"
      image_name: "out_extra_histsampling"



In [82]:
# show the resulting image
ascent.jupyter.AscentViewer(a).show()

In [83]:
# close ascent
a.close()

## Extra Example 4:
### Custom render background and foreground colors

In [85]:
# Add custom background and foreground colors
a = ascent.Ascent()
a.open()

# publish mesh to ascent
a.publish(mesh);

# setup actions
actions = conduit.Node()

# add scene
add_act = actions.append()
add_act["action"] = "add_scenes"
scenes = add_act["scenes"]

scenes["s1/plots/p1/type"] = "pseudocolor"
#scenes["s1/plots/p1/field"] = "radial"
scenes["s1/plots/p1/field"] = "braid"
# specify background and foreground colors in renders
renders = scenes["s1/renders"]
renders["r1/image_prefix"] = "out_extra_bg_fg_color"
# background color
bg = np.array([1.0,1.0,1.0],dtype=np.float32)
renders["r1/bg_color"].set(bg)
# foreground color
fg = np.array([0.0,0.0,0.0],dtype=np.float32)
renders["r1/fg_color"].set(fg)

# print our full actions tree
print(actions.to_yaml())

# execute
a.execute(actions)


- 
  action: "add_scenes"
  scenes: 
    s1: 
      plots: 
        p1: 
          type: "pseudocolor"
          field: "braid"
      renders: 
        r1: 
          image_prefix: "out_extra_bg_fg_color"
          bg_color: [1.0, 1.0, 1.0]
          fg_color: [0.0, 0.0, 0.0]



In [86]:
# show the resulting image
ascent.jupyter.AscentViewer(a).show()

In [54]:
# close ascent
a.close()

## Extra Example 5:
### Trigger with no trigger actions file

In [88]:
# Use triggers to render when conditions occur without actions file
a = ascent.Ascent()
a.open()

# publish mesh to ascent
a.publish(mesh);

# set up actions
actions = conduit.Node()

# declare triggers 
add_triggers = actions.append()
add_triggers["action"] = "add_triggers"
triggers = add_triggers["triggers"] 
triggers["t1/params/condition"] = "magnitude(max(field('braid')).position - vector(0,0,0)) > 0"
# create in-place trigger action
trigger_actions = triggers["t1/params/actions"].append()
# plot when trigger fires
trigger_actions["action"] = "add_scenes"
scenes = trigger_actions["scenes"]
scenes["s1/plots/p1/type"] = "pseudocolor"
#scenes["s1/plots/p1/field"] = "radial"
scenes["s1/plots/p1/field"] = "braid"

# set the output file name (ascent will add ".png")
scenes["s1/image_name"] = "out_extra_inplace_trigger"

# print our full actions tree
print(actions.to_yaml())

# execute
a.execute(actions)


- 
  action: "add_triggers"
  triggers: 
    t1: 
      params: 
        condition: "magnitude(max(field('braid')).position - vector(0,0,0)) > 0"
        actions: 
          - 
            action: "add_scenes"
            scenes: 
              s1: 
                plots: 
                  p1: 
                    type: "pseudocolor"
                    field: "braid"
                image_name: "out_extra_inplace_trigger"



In [89]:
# show the resulting trigger info
ascent.jupyter.AscentViewer(a).show()

In [54]:
# close ascent
a.close()

## Exercise

**First**, copy the code from notebook 8 and paste below. Re-run this code to refresh your memory of the output.

**Second** refactor this code so that you get the same output as notebook 8 but *without* using the trigger files "cycle_trigger_actions.yaml" or "entropy_trigger_actions.yaml".

## Exercise solution
Run the cell below once to see solutions and twice to run them.

In [ ]:
%load solutions/exercise10.py